In [ ]:
import heapq
import copy
import time

class SudokuSolver:
    def __init__(self, board):
        self.board = board
        self.size = 9
        self.box_size = 3
        self.empty_cell = 0

    def is_valid(self, row, col, num):
        for x in range(self.size):
            if self.board[row][x] == num:
                return False

        for x in range(self.size):
            if self.board[x][col] == num:
                return False

        box_row, box_col = (row // self.box_size) * self.box_size, (col // self.box_size) * self.box_size
        for i in range(box_row, box_row + self.box_size):
            for j in range(box_col, box_col + self.box_size):
                if self.board[i][j] == num:
                    return False

        return True

    def find_empty(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == self.empty_cell:
                    return (i, j)
        return None

    def count_empty_cells(self):
        count = 0
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == self.empty_cell:
                    count += 1
        return count

    def get_possible_values(self, row, col):
        possible_values = []
        for num in range(1, 10):
            if self.is_valid(row, col, num):
                possible_values.append(num)
        return possible_values

    def calculate_heuristic(self):
        h_value = 0
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == self.empty_cell:
                    possible_values = self.get_possible_values(i, j)
                    if not possible_values:
                        return float('inf')
                    h_value += len(possible_values)
        return h_value

    def solve_astar(self):
        start_time = time.time()

        initial_board = copy.deepcopy(self.board)
        initial_g = self.size * self.size - self.count_empty_cells()
        initial_h = self.calculate_heuristic()
        initial_f = initial_g + initial_h

        open_set = [(initial_f, initial_board, initial_g)]
        heapq.heapify(open_set)

        closed_set = set()
        states_explored = 0

        while open_set:
            f_value, current_board, g_value = heapq.heappop(open_set)
            states_explored += 1

            board_tuple = tuple(tuple(row) for row in current_board)
            if board_tuple in closed_set:
                continue

            closed_set.add(board_tuple)
            self.board = current_board

            empty_cell = self.find_empty()
            if not empty_cell:
                end_time = time.time()
                print(f"Solution found in {end_time - start_time:.4f} seconds.")
                print(f"States explored: {states_explored}")
                return True

            row, col = empty_cell
            possible_values = self.get_possible_values(row, col)

            for num in possible_values:
                new_board = copy.deepcopy(current_board)
                new_board[row][col] = num

                new_g = g_value + 1

                self.board = new_board
                new_h = self.calculate_heuristic()
                new_f = new_g + new_h

                heapq.heappush(open_set, (new_f, new_board, new_g))

        end_time = time.time()
        print(f"No solution found in {end_time - start_time:.4f} seconds.")
        print(f"States explored: {states_explored}")
        return False

    def print_board(self):
        for i in range(self.size):
            if i % self.box_size == 0 and i != 0:
                print("- - - - - - - - - - - -")
            for j in range(self.size):
                if j % self.box_size == 0 and j != 0:
                    print("| ", end="")
                if j == 8:
                    print(self.board[i][j])
                else:
                    print(str(self.board[i][j]) + " ", end="")

if __name__ == "__main__":
    board = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

    solver = SudokuSolver(board)
    print("Initial board:")
    solver.print_board()

    print("\nSolving...")
    solver.solve_astar()

    print("\nSolution:")
    solver.print_board()

Initial board:
5 3 0 | 0 7 0 | 0 0 0
6 0 0 | 1 9 5 | 0 0 0
0 9 8 | 0 0 0 | 0 6 0
- - - - - - - - - - - -
8 0 0 | 0 6 0 | 0 0 3
4 0 0 | 8 0 3 | 0 0 1
7 0 0 | 0 2 0 | 0 0 6
- - - - - - - - - - - -
0 6 0 | 0 0 0 | 2 8 0
0 0 0 | 4 1 9 | 0 0 5
0 0 0 | 0 8 0 | 0 7 9

Solving...
Solution found in 0.0740 seconds.
States explored: 147

Solution:
5 3 4 | 6 7 8 | 9 1 2
6 7 2 | 1 9 5 | 3 4 8
1 9 8 | 3 4 2 | 5 6 7
- - - - - - - - - - - -
8 5 9 | 7 6 1 | 4 2 3
4 2 6 | 8 5 3 | 7 9 1
7 1 3 | 9 2 4 | 8 5 6
- - - - - - - - - - - -
9 6 1 | 5 3 7 | 2 8 4
2 8 7 | 4 1 9 | 6 3 5
3 4 5 | 2 8 6 | 1 7 9
